In [ ]:
class FirstOrderTransientModel:
    """Class defining a first order implicit transient model"""

    def __init__(self, grid, T, Told, rho, cp, dt):
        """Constructor"""
        self._grid = grid
        self._T = T
        self._Told = Told
        self._rho = rho
        self._cp = cp
        self._dt = dt

    def add(self, coeffs):
        """Function to add transient term to coefficient arrays"""

        # Calculate the transient term
                
        transient = ((self._rho*self._cp*self._grid.vol)*(self._T[1:-1] - self._Told[1:-1]))/self._dt
        
        # Calculate the linearization coefficients
        coeffP = (self._rho*self._cp*self._grid.vol)/self._dt
        
        # Add to coefficient arrays
        coeffs.accumulate_aP(coeffP)
        coeffs.accumulate_rP(transient)

        return coeffs

In [ ]:
class RobinBc:
    """Class defining a Robin boundary condition"""
    
    def __init__(self, phi, grid, T_inf, loc, ha, k):
        """Constructor
            phi ........ field variable array
            grid ....... grid
            T_inf........ Ambient Temperature
            ha is the convective heat transfer coefficient
            loc ........ boundary location
            k.......thermal conductivity
        """
        self._phi = phi
        self._grid = grid
        self._T_inf = T_inf
        self._ha = ha
        self._k = k
        self._loc = loc
        
    def value(self):
        """Return the boundary condition value"""
        if self._loc is BoundaryLocation.WEST:
            return (self._phi[1] + (self._grid.dx_WP[0]*(self._ha/self._k)*self._T_inf))/(1 + (self._grid.dx_WP[0]*(self._ha/self._k)))
        elif self._loc is BoundaryLocation.EAST:
            return ( self._phi[-2] + (self._grid.dx_PE[-1]*(self._ha/self._k)*self._T_inf))/(1 + (self._grid.dx_PE[-1]*(self._ha/self._k)))
        else:
            raise ValueError("Unknown boundary location")
    
    def coeff(self):
        """Return the linearization coefficient"""
        return 1 /(1 + (self._grid.dx_WP[0]*(self._ha/self._k)))
    
    def apply(self):
        """Applies the boundary condition in the referenced field variable array"""
        if self._loc is BoundaryLocation.WEST:
            self._phi[0] = (self._phi[1] + (self._grid.dx_WP[0]*(self._ha/self._k)*self._T_inf))/(1 + (self._grid.dx_WP[0]*(self._ha/self._k)))
        elif self._loc is BoundaryLocation.EAST:
            self._phi[-1] = ( self._phi[-2] + (self._grid.dx_PE[-1]*(self._ha/self._k)*self._T_inf))/(1 + (self._grid.dx_PE[-1]*(self._ha/self._k)))
        else:
            raise ValueError("Unknown boundary location")

In [ ]:
# Grid test 1 for ncv 8
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 8
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 32
dt = 0.0878
time = 0.4535
# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial temperature Function 
T1 = 0
Ti = 100
Zi = 0.8603    #Eigenvalue
C1 = 1.1191    #ConstantC1
L = 1
T2 = (100*(C1*np.exp(-Zi**2*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)         
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha , k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)
        
        
        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()

    
    dt1 = dt
    T2_1 = T[1]
    timeStep_1 = nTime
    T_solns.append(np.copy(T))
    
    #Storing Temperature for Grid Test
    Tg1 = T
    xP1 = grid.xP


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()
plt.show()

In [ ]:
# Grid test 2 for ncv 16
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 32
dt = 0.0878
time = 0.4535
# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial temperature Function 
T1 = 0
Ti = 100
Zi = 0.8603    #Eigenvalue
C1 = 1.1191    #ConstantC1
L = 1
T2 = (100*(C1*np.exp(-Zi**2*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)         
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha , k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)
        
        
        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()

    
    dt1 = dt
    T2_1 = T[1]
    timeStep_1 = nTime
    T_solns.append(np.copy(T))
    
    #Storing Temperature for Grid Test
    Tg2 = T
    xP2 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()
plt.show()

In [ ]:
# Grid test 3 for ncv 32
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 32
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 32
dt = 0.0878
time = 0.4535
# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial temperature Function 
T1 = 0
Ti = 100
Zi = 0.8603    #Eigenvalue
C1 = 1.1191    #ConstantC1
L = 1
T2 = (100*(C1*np.exp(-Zi**2*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)         
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha , k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)
        
        
        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()

    
    dt1 = dt
    T2_1 = T[1]
    timeStep_1 = nTime
    T_solns.append(np.copy(T))
    #Storing Temperature for Grid Test
    Tg3 = T
    xP3 = grid.xP


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()
plt.show()

## Grid Independency Test (First order Implicit scheme)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(xP1, Tg1, label='ncv=8')
plt.plot(xP2, Tg2, label='ncv=16')
plt.plot(xP3, Tg3, label='ncv=32')
plt.xlabel("Distance(x)")
plt.ylabel("Temperature(C)")
plt.legend()
#Display the title
plt.title('Fig1: Temperature vs. Distance')
plt.show()

It is visible from figure 1 that the temperature profile is identical for ncv 8,16 and 32. For the this problem, ncv 16 has been choosen for further comparison.

# First Order Implicit Scheme

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 2
dt = 1.40485
time = 0.4535
# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial temperature Function 
T1 = 0
Ti = 100
Zi = 0.8603    #Eigenvalue
C1 = 1.1191    #ConstantC1
L = 1
T2 = (100*(C1*np.exp(-Zi**2*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)         
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha , k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)
        
        
        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()

    
    dt1 = dt
    T2_1_1 = T[1]
    timeStep_1 = nTime
    T_solns.append(np.copy(T))
    


In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error1 = E_bar

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("Distance(x)")
plt.ylabel("Temperature(C)")
plt.legend()
plt.show()

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 4
dt = 0.702425
time = 0.4535
# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial temperature 
T1 = 0
Ti = 100
Zi = 0.8603    #Eigenvalue
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp(-Zi**2*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha , k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)
        
        
        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
    #Storing the temperature
    dt2 = dt 
    T2_2_1 = T[1]   
    timeStep_2 = nTime
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error2 = E_bar

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 8
dt = 0.3512125
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1


# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial temperature 
T1 = 0
Ti = 100
Zi = 0.8603    #Eigenvalue
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp(-Zi**2*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha , k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)
        
        
        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
     #Storing the temperature of T at x=0 at t=3.2632 for comparison
    dt3 = dt
    T2_3_1 = T[1]
    timeStep_3 = nTime
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error3 = E_bar

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 16
dt = 0.175606
time = 0.4535
value = [0.8047, 1.1559, 1.507, 1.85, 2.209, 2.56, 2.91, 3.2632]
# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1
Texact = []

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial temperature 
T1 = 0
Ti = 100
Zi = 0.8603    #Eigenvalue
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp(-Zi**2*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha , k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)
        
        
        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
    
    #Storing the temperature of T at x=0 at t=3.2632 for comparison
    dt4 = dt
    T2_4_1 = T[1]
    timeStep_4 = nTime
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error4 = E_bar

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 32
dt = 0.0878
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1


# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial temperature 
T1 = 0
Ti = 100
Zi = 0.8603    #Eigenvalue
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp(-Zi**2*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha , k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)
        
        
        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    #Storing the temperature of T at x=0 at t=3.2632 for comparison
    dt5 = dt
    dt1_5 = dt
    T2_5_1 = T[1]
    timeStep_5 = nTime
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error5 = E_bar

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

Avg_Error = [Error1, Error2, Error3, Error4, Error5]
del_T = [dt1, dt2, dt3, dt4, dt5]

# Calculating the slope p for first order transient model
Error_1 = np.log(Error1)
Error_2 = np.log(Error2)
Error_3 = np.log(Error3)
Error_4 = np.log(Error4)
Error_5 = np.log(Error5)

dt_1 = np.log(dt1)
dt_2 = np.log(dt2)
dt_3 = np.log(dt3)
dt_4 = np.log(dt4)
dt_5 = np.log(dt5)

plt.loglog(del_T, Avg_Error)
slope = [(Error_2-Error_1)/(dt_2-dt_1), (Error_3-Error_2)/(dt_3-dt_2), (Error_4-Error_3)/(dt_4-dt_3), (Error_5-Error_4)/(dt_5-dt_4) ]

print('The slope is', slope)

plt.xlabel("del_T")
plt.ylabel("Avg_Error")
plt.show()

## Temperature vs Time Step

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

Time_Step1 = [timeStep_1, timeStep_2, timeStep_3, timeStep_4, timeStep_5]
Temperature1 = [T2_1_1, T2_2_1, T2_3_1, T2_4_1, T2_5_1]
plt.plot(Time_Step1,Temperature1)

plt.plot(timeStep_1, T2_1_1 , 'o', label='Time Step 2')
plt.plot(timeStep_2, T2_2_1 , 'x', label='Time Step 4')
plt.plot(timeStep_3, T2_3_1 , '*', label='Time Step 8')
plt.plot(timeStep_4, T2_4_1 ,'p',  label='Time Step 16')
plt.plot(timeStep_5, T2_5_1 , '.',  label='Time Step 32')

plt.xlabel("Time Step")
plt.ylabel("Temperature")
plt.legend()
plt.show()

## Second order implicit scheme

In [ ]:
class SecondOrderTransientModel:
    """Class defining a first order implicit transient model"""

    def __init__(self, grid, T, Told, Told2, rho, cp, dt):
        """Constructor"""
        self._grid = grid
        self._T = T
        self._Told = Told
        self._Told2 = Told2
        self._rho = rho
        self._cp = cp
        self._dt = dt

    def add(self, coeffs):
        """Function to add transient term to coefficient arrays"""

        # Calculate the transient term
        
        transient = ((self._rho*self._cp*self._grid.vol)*(1.5*self._T[1:-1] - 2*self._Told[1:-1] + 0.5*self._Told2[1:-1]))/self._dt
        
        # Calculate the linearization coefficients
        coeffP = (1.5*(self._rho*self._cp*self._grid.vol))/self._dt
        
        # Add to coefficient arrays
        coeffs.accumulate_aP(coeffP)
        coeffs.accumulate_rP(transient)

        return coeffs

## Grid Independency test (Second order Implicit scheme)

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 8
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 2
dt = 1.40485
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
Told2 = np.copy(T)
transient = SecondOrderTransientModel(grid, T, Told, Told2, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told2[:] = Told[:]
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    #print(T)
    T2_1 = T[1]
    timeStep_1 = nTime
    dt1 = dt
    T_solns.append(np.copy(T))
    
    #Storing Temperature for Grid Test
    Tg1 = T
    xP1 = grid.xP

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 2
dt = 1.40485
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
Told2 = np.copy(T)
transient = SecondOrderTransientModel(grid, T, Told, Told2, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told2[:] = Told[:]
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    #print(T)
    T2_1 = T[1]
    timeStep_1 = nTime
    dt1 = dt
    T_solns.append(np.copy(T))
    
    #Storing Temperature for Grid Test
    Tg2 = T
    xP2 = grid.xP

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 32
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 2
dt = 1.40485
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
Told2 = np.copy(T)
transient = SecondOrderTransientModel(grid, T, Told, Told2, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told2[:] = Told[:]
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    #print(T)
    T1_1 = T[1]
    timeStep_1 = nTime
    dt1 = dt
    T_solns.append(np.copy(T))
    
    #Storing Temperature for Grid Test
    Tg3 = T
    xP3 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(xP1, Tg1, label='ncv=8')
plt.plot(xP2, Tg2, label='ncv=16')
plt.plot(xP3, Tg3, label='ncv=32')
plt.xlabel("Distance(x)")
plt.ylabel("Temperature(C)")
plt.legend()
#Display the title
plt.title('Fig1: Temperature vs. Distance')
plt.show()

## Grid Test (Second order implicit scheme)


In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 2
dt = 1.40485
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
Told2 = np.copy(T)
transient = SecondOrderTransientModel(grid, T, Told, Told2, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told2[:] = Told[:]
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    #print(T)
    T2_1 = T[1]
    timeStep_1 = nTime
    dt1 = dt
    T_solns.append(np.copy(T))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error1 = E_bar
print(E_bar)

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 4
dt = 0.702425
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)


# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
Told2 = np.copy(T)
transient = SecondOrderTransientModel(grid, T, Told, Told2, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told2[:] = Told[:]
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        #Storing The temp for Told2
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    #Storing the temperature at x = 0 
    T2_2 = T[1]
    timeStep_2 = nTime
    dt2 = dt
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error2 = E_bar
print(E_bar)

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 8
dt = 0.3512125
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
Told2 = np.copy(T)
transient = SecondOrderTransientModel(grid, T, Told, Told2, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)


# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told2[:] = Told[:]
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        #Storing The temp for Told2
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    T2_3 = T[1]
    timeStep_3 = nTime
    dt3 = dt
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error3 = E_bar
print(E_bar)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()
plt.show()

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 16
dt = 0.17560625
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1
T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)


# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
Told2 = np.copy(T)
transient = SecondOrderTransientModel(grid, T, Told, Told2, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
#surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told2[:] = Told[:]
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = surfaceConvection.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        #Storing The temp for Told2
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    T2_4 = T[1]
    timeStep_4 = nTime
    dt4 = dt
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error4 = E_bar
print(E_bar)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()
plt.show()

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 32
dt = 0.087803125
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1

T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)
T = T2*np.ones(grid.ncv+2)


# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
Told2 = np.copy(T)
transient = SecondOrderTransientModel(grid, T, Told, Told2, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)


# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told2[:] = Told[:]
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        #Storing The temp for Told2
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    T2_5 = T[1]
    timeStep_5 = nTime
    dt5 = dt
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
    Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)

Error5 = E_bar
print(E_bar)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()
plt.show()

## Error vs dt plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

Avg_Error = [Error1, Error2, Error3, Error4, Error5]
del_T = [dt1, dt2, dt3, dt4, dt5]
slope, intercept, r, p, se = linregress(del_T, Avg_Error)
result = linregress(del_T, Avg_Error)
# Calculating the slope p for second order transient model

Error_1 = np.log(Error1)
Error_2 = np.log(Error2)
Error_3 = np.log(Error3)
Error_4 = np.log(Error4)
Error_5 = np.log(Error5)

dt_1 = np.log(dt1)
dt_2 = np.log(dt2)
dt_3 = np.log(dt3)
dt_4 = np.log(dt4)
dt_5 = np.log(dt5)

plt.loglog(del_T, Avg_Error)
slope_Manual = [(Error_1-Error_2)/(dt_1-dt_2), (Error_2-Error_3)/(dt_2-dt_3), (Error_3-Error_4)/(dt_3-dt_4), (Error_4-Error_5)/(dt_4-dt_5) ]
print(slope)
print('The  slope is', slope_Manual)


plt.xlabel("delT")
plt.ylabel("Error")
plt.show()

## Temperature vs Time Step plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
from scipy.stats import linregress
Time_Step = [timeStep_1, timeStep_2, timeStep_3, timeStep_4, timeStep_5]
Temperature = [T2_1, T2_2, T2_3, T2_4, T2_5]
plt.plot(Time_Step,Temperature)

plt.plot(timeStep_1, T2_1 , 'o', label='Time Step 2')
plt.plot(timeStep_2, T2_2 , 'x', label='Time Step 4')
plt.plot(timeStep_3, T2_3 , '*', label='Time Step 8')
plt.plot(timeStep_4, T2_4 ,'p',  label='Time Step 16')
plt.plot(timeStep_5, T2_5 , '.',  label='Time Step 32')

plt.xlabel("Time Step")
plt.ylabel("Temperature")
plt.legend()
plt.show()

# Comparison between First and Second order implicit Scheme

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
from scipy.stats import linregress

Time_Step = [timeStep_1, timeStep_2, timeStep_3, timeStep_4, timeStep_5]
Temperature = [T2_1, T2_2, T2_3, T2_4, T2_5]

plt.plot(Time_Step,Temperature, label='Second order Implicit scheme')


Time_Step1 = [timeStep_1, timeStep_2, timeStep_3, timeStep_4, timeStep_5]
Temperature1 = [T2_1_1, T2_2_1, T2_3_1, T2_4_1, T2_5_1]

plt.plot(Time_Step1,Temperature1, label='First order Implicit scheme')

plt.xlabel("Time Step")
plt.ylabel("Temperature")
plt.legend()
plt.show()


## Crank Nicolson Scheme

In [ ]:
class DiffusionModel_CrankNicolson1:
    """Class defining a diffusion model"""
    
    def __init__(self, grid, phi, gamma, west_bc, east_bc, w, phi_old):
        """Constructor"""
        self._grid = grid
        self._phi = phi
        self._gamma = gamma
        self._west_bc = west_bc
        self._east_bc = east_bc
        self._w = w
        self._phi_old = phi_old
        
    def add(self, coeffs):
        """Function to add diffusion terms to coefficient arrays"""
        
        # Calculate the west and east face diffusion flux terms for each face
        flux_w = -self._w*self._gamma*self._grid.Aw*(self._phi[1:-1]-self._phi[0:-2])/self._grid.dx_WP
        flux_e = -self._w*self._gamma*self._grid.Ae*(self._phi[2:]-self._phi[1:-1])/self._grid.dx_PE
        flux_w_CN = -( 1 - self._w)*self._gamma*self._grid.Aw*(self._phi_old[1:-1]-self._phi_old[0:-2])/self._grid.dx_WP
        flux_e_CN = -(1- self._w)*self._gamma*self._grid.Ae*(self._phi_old[2:]-self._phi_old[1:-1])/self._grid.dx_PE
       
        # Calculate the linearization coefficients
        coeffW = - self._w*self._gamma*self._grid.Aw/self._grid.dx_WP
        coeffE = - self._w*self._gamma*self._grid.Ae/self._grid.dx_PE
        coeffP = ( - coeffW - coeffE)
        
        # Modify the linearization coefficients on the boundaries
        coeffP[0] += coeffW[0]*self._west_bc.coeff()
        coeffP[-1] += coeffE[-1]*self._east_bc.coeff()
        
        # Zero the boundary coefficients that are not used
        coeffW[0] = 0.0
        coeffE[-1] = 0.0
        
        # Calculate the net flux from each cell
        flux = (flux_e - flux_w) + (flux_e_CN - flux_w_CN)
        
        # Add to coefficient arrays
        coeffs.accumulate_aP(coeffP)
        coeffs.accumulate_aW(coeffW)
        coeffs.accumulate_aE(coeffE)
        coeffs.accumulate_rP(flux)
        
        # Return the modified coefficient array
        return coeffs
    

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 10
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 2
dt = 1.40485
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1
w = 0.5
# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1

T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)

T = T2*np.ones(grid.ncv+2)


# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel_CrankNicolson1(grid, T, k, west_bc, east_bc, w , Told)

#Flux = FirstOrder_CrankNicolsonModel(grid, T, k, west_bc, east_bc, w, Told)

# Loop through all timesteps
    

for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
       # coeffs = Flux.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        #Storing The temp for Told2
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    print(T)
    T2_1_CN = T[1]
    timeStep_1 = nTime
    T_solns.append(np.copy(T))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  Tex[1:-1] - T[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)
dt1 = dt
Error1 = E_bar
print(E_bar)

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 10
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 4
dt = 0.702425
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1
w = 0.5
# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1

T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)

T = T2*np.ones(grid.ncv+2)


# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel_CrankNicolson1(grid, T, k, west_bc, east_bc, w , Told)

#Flux = FirstOrder_CrankNicolsonModel(grid, T, k, west_bc, east_bc, w, Told)

# Loop through all timesteps
    

for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = Flux.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        #Storing The temp for Told2
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    print(T)
    dt2 = dt
    T2_2_CN = T[1]
    timeStep_2 = nTime
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  Tex[1:-1] - T[1:-1]
E_bar = sum(Error)/grid.ncv
print(E_bar)
#dt2 = dt
Error2 = E_bar

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 10
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 8
dt = 0.3512125
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1
w = 0.5
# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1

T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)

T = T2*np.ones(grid.ncv+2)


# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel_CrankNicolson1(grid, T, k, west_bc, east_bc, w , Told)

#Flux = FirstOrder_CrankNicolsonModel(grid, T, k, west_bc, east_bc, w, Told)

# Loop through all timesteps
    

for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = Flux.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        #Storing The temp for Told2
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    print(T)
    dt3 = dt
    T2_3_CN = T[1]
    timeStep_3 = nTime
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  Tex[1:-1] - T[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)
dt3 = dt
Error3 = E_bar
print(E_bar)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 10
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 16
dt = 0.17560625
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1
w = 0.5
# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1

T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)

T = T2*np.ones(grid.ncv+2)


# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel_CrankNicolson1(grid, T, k, west_bc, east_bc, w , Told)

#Flux = FirstOrder_CrankNicolsonModel(grid, T, k, west_bc, east_bc, w, Told)

# Loop through all timesteps
    

for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = Flux.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        #Storing The temp for Told2
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    print(T)
    dt4 = dt
    T2_4_CN = T[1]
    timeStep_4 = nTime
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2632))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  Tex[1:-1] - T[1:-1]
E_bar = sum(Error)/grid.ncv
print(Tex)
dt4 = dt
print(dt4)
Error4 = E_bar
print(E_bar)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()

In [ ]:
from Lesson3.Grid import Grid
from Lesson3.ScalarCoeffs import ScalarCoeffs
from Lesson3.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Lesson3.Models import DiffusionModel, SurfaceConvectionModel
from Lesson3.LinearSolver import solve

import numpy as np
from numpy.linalg import norm
import math

# Define the grid
lx = 1.0
ly = 1
lz = 1
ncv = 10
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 32
dt = 0.0878031
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 10
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
ha = 1
w = 0.5
# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions

T1 = 0
Ti = 100
Zi = 0.8603
C1 = 1.1191
L = 1

T2 = (100*(C1*np.exp((-Zi**2)*(time))*np.cos(Zi*grid.xP/L)) + T1)

T = T2*np.ones(grid.ncv+2)


# Define boundary conditions
west_bc = NeumannBc(T, grid, 0, BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, 0, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
# Note: If there are a lot of timesteps, this will cost a
#       lot of memory. It is suggested to set a parameter to 
#       only store the solution every N timesteps.
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel_CrankNicolson1(grid, T, k, west_bc, east_bc, w , Told)

#Flux = FirstOrder_CrankNicolsonModel(grid, T, k, west_bc, east_bc, w, Told)

# Loop through all timesteps
    

for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    # Note: do not use copy here because that creates a new object
    #       and we want the reference in the transient model to remain
    #       valid
    Told[:] = T[:]
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        #coeffs = Flux.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
        #Storing The temp for Told2
          
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
        
    # Store the solution
    print(T)
    T2_5_CN = T[1]
    timeStep_5 = nTime
    T_solns.append(np.copy(T))

In [ ]:
Tex = np.ones(grid.ncv+2)
for i in range(ncv+2):
    Tex[i] = ((100*(1.1191*np.exp(-0.8603**2*(3.2635))*np.cos(0.8603*grid.xP[i]/L)))) 
Error =  T[1:-1] - Tex[1:-1]
E_bar = sum(Error)/grid.ncv
print(E_bar)
print(Tex)
dt5 = dt
Error5 = E_bar

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1
print(T)
plt.xlabel("x")
plt.ylabel("T")
plt.legend()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
from scipy import stats

Avg_Error = [Error1, Error2, Error3, Error4, Error5]
del_T = [dt1, dt2, dt3, dt4, dt5]

slope, intercept, r_value, p_value, std_err = stats.linregress(del_T, Avg_Error)
#res = stats.linregress(del_T, Avg_Error)
#plt.loglog(del_T, res.intercept + res.slope*del_T, 'r', label='fitted line')
# Calculating the slope p for second order transient model
Error_1 = np.log(Error1)
Error_2 = np.log(Error2)
Error_3 = np.log(Error3)
Error_4 = np.log(Error4)
Error_5 = np.log(Error5)

dt_1 = np.log(dt1)
dt_2 = np.log(dt2)
dt_3 = np.log(dt3)
dt_4 = np.log(dt4)
dt_5 = np.log(dt5)

plt.loglog(del_T, Avg_Error)
slope_M = [(Error_2-Error_1)/(dt_2-dt_1), (Error_3-Error_2)/(dt_3-dt_2), (Error_4-Error_3)/(dt_4-dt_3),(Error_5-Error_4)/(dt_4-dt_5) ]

print('The slope is', slope_M)
print(slope)

plt.xlabel("delT")
plt.ylabel("Error")
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
from scipy.stats import linregress

Time_Step = [timeStep_1, timeStep_2, timeStep_3, timeStep_4, timeStep_5]
Temperature_CN = [T2_1_CN, T2_2_CN, T2_3_CN, T2_4_CN, T2_5_CN]

plt.plot(Time_Step,Temperature_CN, label='Crank Nicolson scheme')
plt.xlabel("Time Step")
plt.ylabel("Temperature")
plt.legend()
plt.show()

# Comparison of all the schemes

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
from scipy.stats import linregress


Time_Step1 = [timeStep_1, timeStep_2, timeStep_3, timeStep_4, timeStep_5]
Temperature1 = [T2_1_1, T2_2_1, T2_3_1, T2_4_1, T2_5_1]

plt.plot(Time_Step1,Temperature1, label='First order Implicit scheme')

Time_Step = [timeStep_1, timeStep_2, timeStep_3, timeStep_4, timeStep_5]
Temperature = [T2_1, T2_2, T2_3, T2_4, T2_5]

plt.plot(Time_Step,Temperature, label='Second order Implicit scheme')


Time_Step = [timeStep_1, timeStep_2, timeStep_3, timeStep_4, timeStep_5]
Temperature_CN = [T2_1_CN, T2_2_CN, T2_3_CN, T2_4_CN, T2_5_CN]

plt.plot(Time_Step,Temperature_CN, label='Crank Nicolson scheme')

plt.xlabel("Time Step")
plt.ylabel("Temperature")
plt.legend()
plt.show()